In [7]:
import os
import json
import glob
import numpy as np

# Settings
base_path = "F:/setup files/downloads/ResultLastly/"
output_json_path = os.path.join(base_path, "pareto_fronts_PFG-2F-RK.json")

# Use a wildcard pattern to find all relevant JSON files
file_pattern = os.path.join(base_path, "L*.json")
files = glob.glob(file_pattern)

def is_dominated(point, others):
    """
    Check if 'point' is dominated by any point in 'others'.
    A point p is dominated by q if q is no worse in all objectives and better in at least one.
    """
    return np.any(np.all(others <= point, axis=1) & np.any(others < point, axis=1))

def compute_pareto_front(solutions):
    """
    Compute the Pareto front from a set of solutions.
    Each solution is assumed to be a numpy array of objective values.
    """
    if len(solutions) == 0:
        return np.array([])
    
    solutions = np.array(solutions)
    pareto_mask = np.ones(solutions.shape[0], dtype=bool)
    
    for i, sol in enumerate(solutions):
        if pareto_mask[i]:
            # If any other solution dominates this one, mark as non-efficient
            pareto_mask[i] = not is_dominated(sol, solutions[np.arange(len(solutions)) != i])
            
            # Also remove any solutions dominated by this one
            for j in range(i+1, len(solutions)):
                if pareto_mask[j] and np.all(sol <= solutions[j]) and np.any(sol < solutions[j]):
                    pareto_mask[j] = False
    
    return solutions[pareto_mask]

pareto_fronts = {}

for file_path in files:
    file_name = os.path.basename(file_path)
    
    # Load JSON data from file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Gather all solutions at generation 100 across all algorithms
    all_solutions_gen100 = []
    for algorithm in ["PFG-EMOA"]:
        gen_key = "100"
        if gen_key in data[algorithm]:
            solutions = data[algorithm][gen_key]
            # Ensure solutions are numeric arrays
            all_solutions_gen100.extend(solutions)
    
    # Compute Pareto front from all gathered solutions
    pareto_front = compute_pareto_front(all_solutions_gen100)
    
    # Store the Pareto front (convert to list for JSON serialization)
    pareto_fronts[file_name] = pareto_front.tolist()


# Save all computed Pareto fronts to a JSON file
with open(output_json_path, 'w') as outfile:
    json.dump(pareto_fronts, outfile, indent=2)

print(f"Global Pareto fronts at generation 100 saved to {output_json_path}")

Global Pareto fronts at generation 100 saved to F:/setup files/downloads/ResultLastly/pareto_fronts_PFG-2F-RK.json


In [18]:
import os
import json
import pandas as pd
import numpy as np

# -------------------- Configuration -------------------- #

# Define the base directory where your files are located
base_path = "F:/setup files/downloads/ResultLastly/"

# Path to the Pareto fronts JSON file
pareto_json_filename = "pareto_fronts_PFG-2F-RK.json"
pareto_json_path = os.path.join(base_path, pareto_json_filename)

# Path to the single-objective results CSV file
single_result_csv_filename = "FairGA.csv"  # Replace with your actual CSV filename
single_result_csv_path = os.path.join(base_path, single_result_csv_filename)

# Path to save the output CSV file
output_csv_filename = "pareto_vs_single_domination_counts.csv"
output_csv_path = os.path.join(base_path, output_csv_filename)

# -------------------- Load Pareto Fronts -------------------- #

# Load Pareto fronts from JSON
with open(pareto_json_path, 'r') as f:
    pareto_fronts = json.load(f)

print(f"Loaded Pareto fronts from {pareto_json_path}")

# -------------------- Load Single-Objective Results -------------------- #

# Load single-objective results from CSV
single_results = pd.read_csv(single_result_csv_path)

print(f"Loaded single-objective results from {single_result_csv_path}")

# Verify the required columns exist
required_columns = ['Instance', 'EC', 'VF', 'CF', 'MWT']  # Adjust 'CF1', 'CF2' as per your CSV
for col in required_columns:
    if col not in single_results.columns:
        raise ValueError(f"Missing required column '{col}' in the CSV file.")

# -------------------- Define Dominance Function -------------------- #

def dominates(pareto_solution, single_solution):
    """
    Check if a Pareto solution dominates the single solution.
    Dominance is defined as being larger or equal in all objectives.
    
    Parameters:
    - pareto_solution: numpy array of Pareto solution objectives
    - single_solution: numpy array of single solution objectives
    
    Returns:
    - Boolean indicating whether pareto_solution dominates single_solution
    """
    return np.all(pareto_solution >= single_solution)

# -------------------- Compare and Count Dominations -------------------- #

# Initialize a list to store domination counts
domination_counts = []

# Iterate over each row in the single-objective results
for idx, row in single_results.iterrows():
    instance = row['Instance']
    
    # Extract single solution objectives
    # Adjust the order and names based on your specific objectives
    single_solution = np.array([row['EC'], row['VF'], row['CF'], row['MWT']])
    
    # Retrieve the corresponding Pareto front
    print(instance[:-4] + ".json")
    pareto_front = pareto_fronts.get(instance[:-4] + "_0.json", [])

    
    if not pareto_front:
        print(f"Warning: No Pareto front found for instance '{instance}'.")
        domination_count = 0
    else:
        # Convert Pareto front to numpy array for vectorized operations
        pareto_front_array = np.array(pareto_front)

        print(pareto_front_array)
        print(single_solution)
        
        # Check dominance for each Pareto solution
        # Assuming all objectives are to be maximized
        dominance = np.all(pareto_front_array <= single_solution, axis=1)
        
        # Count the number of dominating solutions
        domination_count = np.sum(dominance)

        # raise Exception
    
    # Append the result
    domination_counts.append({
        'Instance': instance,
        'Dominating_Solutions_Count': int(domination_count)
    })

# -------------------- Save Results -------------------- #

# Create a DataFrame from the results
domination_df = pd.DataFrame(domination_counts)

# Save the domination counts to a CSV file
domination_df.to_csv(output_csv_path, index=False)

print(f"Domination counts saved to {output_csv_path}")

Loaded Pareto fronts from F:/setup files/downloads/ResultLastly/pareto_fronts_PFG-2F-RK.json
Loaded single-objective results from F:/setup files/downloads/ResultLastly/FairGA.csv
LC1_2_1.json
[[1103.14647867   21.2046039   161.73760932  407.11287923]
 [1373.51232681    3.42365433  187.5350048   386.31189394]
 [1118.163826     21.53379093  142.22369643  332.70675971]
 [1156.57014749    8.47604222  109.08641018  261.06410114]
 [1370.63371143    3.27073497  182.76842373  440.49703273]
 [1316.20959968    3.49096451  174.13818645  440.49703273]
 [1137.546534     16.46746635  164.13738187  399.74841644]
 [1232.43981281    3.64396427  137.44785096  376.24575465]
 [1371.10381564    3.31957716  186.7619071   419.82957575]
 [1134.45907424   18.76669868  134.25693968  323.68362766]
 [1128.42722845   12.09021092  158.86392103  407.11287923]
 [1134.38764788   11.74726774  155.51751363  407.11287923]
 [1134.01256352   19.90191574  168.17112274  399.74841644]
 [1211.70447267    4.08388772  142.918242

In [19]:
import os
import json
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean

# -------------------- Configuration -------------------- #

# Define the base directory where your files are located
base_path = "F:/setup files/downloads/ResultLastly/"

# Path to the Pareto fronts JSON file
pareto_json_filename = "pareto_fronts_PFG-2F-RK.json"
pareto_json_path = os.path.join(base_path, pareto_json_filename)

# Path to the single-objective results CSV file
single_result_csv_filename = "FairGA.csv"  # Replace with your actual CSV filename
single_result_csv_path = os.path.join(base_path, single_result_csv_filename)

# Path to save the output CSV file
output_improvement_csv_filename = "pareto_knee_improvements.csv"
output_improvement_csv_path = os.path.join(base_path, output_improvement_csv_filename)

# -------------------- Load Pareto Fronts -------------------- #

# Load Pareto fronts from JSON
with open(pareto_json_path, 'r') as f:
    pareto_fronts = json.load(f)

print(f"Loaded Pareto fronts from {pareto_json_path}")

# -------------------- Load Single-Objective Results -------------------- #

# Load single-objective results from CSV
single_results = pd.read_csv(single_result_csv_path)

print(f"Loaded single-objective results from {single_result_csv_path}")

# Verify the required columns exist
required_columns = ['Instance', 'EC', 'VF', 'CF', 'MWT']  # Adjust as per your CSV
for col in required_columns:
    if col not in single_results.columns:
        raise ValueError(f"Missing required column '{col}' in the CSV file.")

# -------------------- Define Dominance Function -------------------- #

def dominates(pareto_solution, single_solution):
    """
    Check if a Pareto solution dominates the single solution.
    Dominance is defined as being less than or equal in all objectives.
    
    Parameters:
    - pareto_solution: numpy array of Pareto solution objectives
    - single_solution: numpy array of single solution objectives
    
    Returns:
    - Boolean indicating whether pareto_solution dominates single_solution
    """
    return np.all(pareto_solution <= single_solution)

# -------------------- Define Knee Point Selection Function -------------------- #

def select_knee_point(dominated_solutions, single_solution):
    """
    Select the knee point from dominated solutions using a heuristic method.
    The knee point is chosen as the solution with the maximum Euclidean distance from the ideal point.
    
    Parameters:
    - dominated_solutions: numpy array of solutions that dominate the single solution
    - single_solution: numpy array of single solution objectives
    
    Returns:
    - knee_point: numpy array representing the selected knee point
    """
    if dominated_solutions.shape[0] == 0:
        return None  # No dominating solutions

    # Normalize objectives
    # Assuming all objectives are to be minimized
    # Normalization: (value - min) / (max - min)
    min_vals = dominated_solutions.min(axis=0)
    max_vals = dominated_solutions.max(axis=0)
    ranges = max_vals - min_vals
    # Avoid division by zero
    ranges[ranges == 0] = 1
    normalized = (dominated_solutions - min_vals) / ranges

    # Ideal point (minimum for all objectives)
    ideal_point = np.min(normalized, axis=0)

    # Calculate Euclidean distance from the ideal point
    distances = np.linalg.norm(normalized - ideal_point, axis=1)

    # Select the solution with the maximum distance
    knee_index = np.argmax(distances)
    knee_point = dominated_solutions[knee_index]

    return knee_point

# -------------------- Compare, Select Knee Point, and Calculate Improvements -------------------- #

# Initialize a list to store improvement results
improvement_results = []

# Iterate over each row in the single-objective results
for idx, row in single_results.iterrows():
    instance = row['Instance']
    
    # Extract single solution objectives
    single_solution = np.array([row['EC'], row['VF'], row['CF'], row['MWT']])
    
    # Retrieve the corresponding Pareto front
    # Adjust the key based on how instances are named in the JSON
    # Here, assuming the JSON key is 'instance_name_0.json'
    json_key = instance[:-4] + "_0.json"  # Remove file extension and append '_0.json'
    pareto_front = pareto_fronts.get(json_key, [])
    
    if not pareto_front:
        print(f"Warning: No Pareto front found for instance '{instance}'.")
        improvement = {
            'Instance': instance,
            'EC_Improvement (%)': np.nan,
            'VF_Improvement (%)': np.nan,
            'CF_Improvement (%)': np.nan,
            'MWT_Improvement (%)': np.nan
        }
    else:
        # Convert Pareto front to numpy array
        pareto_front_array = np.array(pareto_front)
        
        # Identify dominated solutions
        dominated_mask = np.array([dominates(sol, single_solution) for sol in pareto_front_array])
        dominated_solutions = pareto_front_array[dominated_mask]
        
        if dominated_solutions.shape[0] == 0:
            print(f"No dominating solutions found for instance '{instance}'.")
            improvement = {
                'Instance': instance,
                'EC_Improvement (%)': 0.0,
                'VF_Improvement (%)': 0.0,
                'CF_Improvement (%)': 0.0,
                'MWT_Improvement (%)': 0.0
            }
        else:
            # Select knee point
            knee_point = select_knee_point(dominated_solutions, single_solution)
            
            if knee_point is None:
                print(f"No knee point could be selected for instance '{instance}'.")
                improvement = {
                    'Instance': instance,
                    'EC_Improvement (%)': np.nan,
                    'VF_Improvement (%)': np.nan,
                    'CF_Improvement (%)': np.nan,
                    'MWT_Improvement (%)': np.nan
                }
            else:
                # Calculate improvement percentages
                # Since objectives are minimized, improvement = ((single - pareto) / single) * 100
                # Handle division by zero if any single objective is zero
                with np.errstate(divide='ignore', invalid='ignore'):
                    improvements = ((single_solution - knee_point) / single_solution) * 100
                    improvements = np.where(single_solution == 0, 0, improvements)  # Replace inf with 0
                
                improvement = {
                    'Instance': instance,
                    'EC_Improvement (%)': round(improvements[0], 2),
                    'VF_Improvement (%)': round(improvements[1], 2),
                    'CF_Improvement (%)': round(improvements[2], 2),
                    'MWT_Improvement (%)': round(improvements[3], 2)
                }
    
    # Append the result
    improvement_results.append(improvement)

# -------------------- Save Improvement Results -------------------- #

# Create a DataFrame from the results
improvement_df = pd.DataFrame(improvement_results)

# Save the improvement percentages to a CSV file
improvement_df.to_csv(output_improvement_csv_path, index=False)

print(f"Improvement percentages saved to {output_improvement_csv_path}")

Loaded Pareto fronts from F:/setup files/downloads/ResultLastly/pareto_fronts_PFG-2F-RK.json
Loaded single-objective results from F:/setup files/downloads/ResultLastly/FairGA.csv
Improvement percentages saved to F:/setup files/downloads/ResultLastly/pareto_knee_improvements.csv


In [20]:
import os
import pandas as pd
import numpy as np
import re

# -------------------- Configuration -------------------- #

# Define the base directory where your files are located
base_path = "F:/setup files/downloads/ResultLastly/"

# Path to the improvement CSV file
improvement_csv_filename = "pareto_knee_improvements.csv"  # Replace with your actual CSV filename
improvement_csv_path = os.path.join(base_path, improvement_csv_filename)

# Path to save the group average CSV file
output_group_avg_csv_filename = "group_average_improvements.csv"
output_group_avg_csv_path = os.path.join(base_path, output_group_avg_csv_filename)

# Define the group patterns
group_patterns = ['LRC1', 'LRC2', 'LC1', 'LC2', 'LR1', 'LR2']

# -------------------- Load Improvement Data -------------------- #

# Load the improvement data from CSV
try:
    improvement_df = pd.read_csv(improvement_csv_path)
    print(f"Loaded improvement data from {improvement_csv_path}")
except FileNotFoundError:
    raise FileNotFoundError(f"The file {improvement_csv_path} does not exist.")

# Verify the required columns exist
required_columns = ['Instance', 'EC_Improvement (%)', 'VF_Improvement (%)', 
                    'CF_Improvement (%)', 'MWT_Improvement (%)']
for col in required_columns:
    if col not in improvement_df.columns:
        raise ValueError(f"Missing required column '{col}' in the CSV file.")

# -------------------- Assign Groups to Instances -------------------- #

def assign_group(instance_name, patterns):
    """
    Assigns a group to an instance based on predefined patterns.

    Parameters:
    - instance_name: str, name of the instance
    - patterns: list of str, group patterns to match

    Returns:
    - group: str or None, the matched group or None if no match
    """
    for pattern in patterns:
        if re.search(pattern, instance_name, re.IGNORECASE):
            return pattern
    return None  # or 'Unknown' if you prefer

# Apply the group assignment
improvement_df['Group'] = improvement_df['Instance'].apply(lambda x: assign_group(x, group_patterns))

# Optional: Handle instances that do not match any group
unmatched = improvement_df[improvement_df['Group'].isnull()]
if not unmatched.empty:
    print("Warning: Some instances did not match any group patterns:")
    print(unmatched['Instance'].unique())
    # Optionally, you can assign them to a group like 'Unknown'
    # improvement_df['Group'].fillna('Unknown', inplace=True)

# Remove instances without a group if desired
improvement_df = improvement_df[improvement_df['Group'].notnull()]

# -------------------- Calculate Average Improvements -------------------- #

# Define the objective columns for averaging
objective_columns = ['EC_Improvement (%)', 'VF_Improvement (%)', 
                    'CF_Improvement (%)', 'MWT_Improvement (%)']

# Group by 'Group' and calculate the mean for each objective
group_avg_df = improvement_df.groupby('Group')[objective_columns].mean().reset_index()

# Round the averages to two decimal places for readability
group_avg_df[objective_columns] = group_avg_df[objective_columns].round(2)

# -------------------- Save Group Averages to CSV -------------------- #

# Save the group average improvements to a CSV file
group_avg_df.to_csv(output_group_avg_csv_path, index=False)

print(f"Group average improvements saved to {output_group_avg_csv_path}")

# -------------------- Optional: Display the Results -------------------- #

print("\nAverage Improvements per Group:")
print(group_avg_df)


Loaded improvement data from F:/setup files/downloads/ResultLastly/pareto_knee_improvements.csv
Group average improvements saved to F:/setup files/downloads/ResultLastly/group_average_improvements.csv

Average Improvements per Group:
  Group  EC_Improvement (%)  VF_Improvement (%)  CF_Improvement (%)  \
0   LC1               15.42               69.15               39.74   
1   LC2               18.74               58.04               45.75   
2   LR1               16.29               64.83               28.85   
3   LR2               14.56               59.95               58.11   
4  LRC1               17.62               64.90               31.35   
5  LRC2               15.85               64.40               58.87   

   MWT_Improvement (%)  
0                40.11  
1                36.18  
2                34.46  
3                49.19  
4                33.60  
5                52.21  
